In [ ]:
import cv2
import pytesseract
import numpy as np
from PIL import Image, ImageDraw

In [ ]:
def load_image(path):
    """Image loading function for images."""
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)


def resize_image(image, target_size):
    """
    Resize image while maintaining original aspect ratio.

    Parameters
    ----------
    image : numpy.ndarray
        Image with 'channel_last' format.
    target_size : int
        Target image size.

    Returns
    -------
    numpy.ndarray
        Output image.
    """
    h, w = image.shape[:2]
    scale = target_size / max(h, w)
    size = (round(scale * w), round(scale * h))
    return cv2.resize(image, size, interpolation=cv2.INTER_CUBIC)


def improve_image_sharpness(image):
    """Improve sharpness of image with unsharp masking.

    Parameters
    ----------
    image : numpy.ndarray
        Image with 'channel_last' format.

    Returns
    -------
    numpy.ndarray
        Output image.

    """
    blurred_image = cv2.GaussianBlur(image, (9, 9), 10)
    return cv2.addWeighted(image, 1.5, blurred_image, -0.5, 0)


def improve_image_contrast(image):
    """Improve contrast of image with contrast limited adaptive
    histogram equalization (CLAHE).

    Parameters
    ----------
    image : numpy.ndarray
        Image with 'channel_last' format.

    Returns
    -------
    numpy.ndarray
        Output image.

    """
    clahe = cv2.createCLAHE(clipLimit=4.0)
    h, s, v = cv2.split(cv2.cvtColor(image, cv2.COLOR_RGB2HSV))
    v = clahe.apply(v)
    return cv2.cvtColor(cv2.merge([h, s, v]), cv2.COLOR_HSV2RGB)


def pad_image(image):
    """Pad image with zero pixel values into square size image.

    Parameters
    ----------
    image : numpy.ndarray
        A RGB image with image format, 'channel_last'.

    Returns
    -------
    numpy.ndarray
        A padded squared RGB image with image format, 'channel_last'.

    """
    h, w = image.shape[:2]

    if h == w:
        return image

    diff_h = max(w - h, 0)
    pad_h = int(diff_h / 2)

    diff_w = max(h - w, 0)
    pad_w = int(diff_w / 2)

    pad_width = (
        (pad_h, diff_h - pad_h),
        (pad_w, diff_w - pad_w),
        (0, 0),
    )

    return np.pad(image, pad_width, "constant", constant_values=255)


def preprocess_image(image, target_size=None, improve_sharpness=True, improve_contrast=True, add_padding=True):
    """
    Preprocess image

    Parameters
    ----------
    image : [type]
        [description]
    target_size : [type], optional
        [description], by default None
    improve_sharpness : bool, optional
        [description], by default True
    improve_contrast : bool, optional
        [description], by default True

    Returns
    -------
    [type]
        [description]
    """
    """Get proprocessed images."""
    try:
        if target_size:
            image = resize_image(image, target_size)

        # improve image sharpness
        if improve_sharpness:
            image = improve_image_sharpness(image)

        # improve image contrast
        if improve_contrast:
            image = improve_image_contrast(image)

        if add_padding:
            # pad image into square shape
            image = pad_image(image)

        return image

    except Exception as e:
        print(e)
        return None



In [ ]:
img_path = "data/Own-Samples/AA 2018_30_crop.jpg"

In [ ]:
img = Image.open(img_path).convert("RGB")
print(img.size)

draw = ImageDraw.Draw(img)
boxes = pytesseract.image_to_data(img)

for b in boxes.splitlines()[1:]:
    b = b.split()
    if len(b) == 12 and float(b[10])>0:
        x, y, w, h = int(b[6]), int(b[7]), int(b[8]), int(b[9])
        draw.rectangle(((x, y), (x+w, y+h)), outline="red", width=2)

img

In [ ]:
img = load_image(img_path)
img = preprocess_image(img, target_size=1500)
img = Image.fromarray(img)
img.save('2.jpg')
print(img.size)

draw = ImageDraw.Draw(img)
boxes = pytesseract.image_to_data(img)

for b in boxes.splitlines()[1:]:
    b = b.split()
    if len(b) == 12 and float(b[10])>0:
        x, y, w, h = int(b[6]), int(b[7]), int(b[8]), int(b[9])
        draw.rectangle(((x, y), (x+w, y+h)), outline="red", width=2)

img